In [1]:
import os
from plant_disease_classification import logger
os.chdir("../")
%pwd

'/home/fibonacci/projects/Plant_Disease_Classification'

In [3]:
import tensorflow as tf
model=tf.keras.models.load_model("artifacts/training/model.h5")

2024-07-22 14:10:48,805 WARNING  Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [10]:
from plant_disease_classification.utils.common import read_yaml,create_directories,save_json
from plant_disease_classification.constants import *

In [12]:
class ConfigurationManager:
    def __init__ (self,config=CONFIGURATION_FILE_PATH,params=PARAMS_FILE_PATH):
        self.config = read_yaml(config)
        self.params=read_yaml(params)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self)->EvaluationConfig:
        eval_config=EvaluationConfig(
            path_of_model=Path(self.config.artifacts_root, "training", "model.h5"),
            training_data='artifacts/data_ingestion/data/',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE[:-1],
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [8]:

from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size,
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)

    def save_score(self):
        scores={"loss":self.score[0],'accuracy':self.score[1]}
        print(scores)
        #save_json(path=Path('scores.json'),content=scores)


In [15]:
try:
    config_manager=ConfigurationManager()
    eval_config=config_manager.get_validation_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    logger.error(f"Error in validation: {str(e)}")

2024-07-22 15:30:57,016 INFO  yaml file:config/config.yaml load successfully


2024-07-22 15:30:57,075 INFO  yaml file:params.yaml load successfully
2024-07-22 15:30:57,088 INFO  Creating directory: artifacts
2024-07-22 15:30:58,227 WARNING  Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Found 870 images belonging to 33 classes.


/home/fibonacci/projects/Plant_Disease_Classification/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


28/28 ━━━━━━━━━━━━━━━━━━━━ 1391s 50s/step - accuracy: 0.5957 - loss: 1.7781
2024-07-22 15:54:11,045 ERROR  Error in validation: save_json() got an unexpected keyword argument 'content'
